Задача: нужно написать код на Python, модель с использованием генетического алгоритма для оптимизации цен на товары с целью максимизации прибыли.

In [2]:
import math
import random
import pandas as pd
import numpy as np
from pathlib import Path

# === ПАРАМЕТРЫ МОДЕЛИ ===
CSV_PATH = "retail_price.csv"   # путь к вашему файлу
POP_SIZE = 200                  # размер популяции (число "ботов")
GENERATIONS = 80                # число поколений
TOURNAMENT_K = 3                # размер турнира при отборе
CXPB = 0.6                      # вероятность кроссовера
MUTPB = 0.3                     # вероятность мутации
MUT_STD = 0.1                   # стандартное отклонение мутации
MULT_MIN, MULT_MAX = 0.5, 2.0   # диапазон множителя (от 50% до 200%)
ELASTICITY = -1.0               # эластичность спроса (по умолчанию -1)
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)


# === ПОМОЩНИКИ ===
def find_columns(df):
    """Автоматический поиск колонок qty, unit_price, freight_price."""
    names = {c.lower(): c for c in df.columns}
    def get_like(key_words):
        for kw in key_words:
            for k,v in names.items():
                if kw in k:
                    return v
        return None
    qty_col = get_like(['qty', 'quantity', 'volume', 'units'])
    unit_price_col = get_like(['unit_price','unit price','price','retail'])
    total_price_col = get_like(['total_price','total price','total'])
    freight_col = get_like(['freight','cost','freight_price','wholesale','purchase'])
    return qty_col, unit_price_col, total_price_col, freight_col


def safe_numeric(s):
    """Приводим строковые значения к числу с плавающей точкой."""
    try:
        if pd.isna(s):
            return np.nan
        if isinstance(s, (int,float)):
            return float(s)
        s = str(s).strip().replace(',', '.').replace(' ', '')
        return float(s)
    except:
        return np.nan


def evaluate_multiplier(df, multiplier, elasticity=ELASTICITY):
    """Оценка прибыли при данном множителе."""
    p_old = df['unit_price'].to_numpy(dtype=float)
    q_old = df['qty'].to_numpy(dtype=float)
    freight = df['freight_price'].to_numpy(dtype=float)

    # Новая цена
    p_new = p_old * multiplier

    # Модель спроса: Q_new = Q_old * (P_new/P_old)^elasticity
    with np.errstate(divide='ignore', invalid='ignore'):
        ratio = np.where(p_old == 0, 1.0, p_new / p_old)
        q_new = q_old * (ratio ** elasticity)

    q_new = np.where(np.isfinite(q_new), q_new, 0.0)
    q_new = np.maximum(q_new, 0.0)

    # Прибыль = (новая цена - закупка) * количество
    profit_per_row = (p_new - freight) * q_new
    profit_per_row = np.where(np.isfinite(profit_per_row), profit_per_row, 0.0)
    total_profit = np.sum(profit_per_row)

    return float(total_profit), p_new, q_new, profit_per_row


# === ЗАГРУЗКА CSV ===
df = pd.read_csv('drive/MyDrive/Нейросети/retail_price.csv', sep=',')
qty_col, unit_price_col, total_price_col, freight_col = find_columns(df)

if not all([qty_col, unit_price_col, freight_col]):
    raise ValueError("Не удалось найти необходимые колонки в CSV")

df['qty'] = df[qty_col].apply(safe_numeric)
df['unit_price'] = df[unit_price_col].apply(safe_numeric)
df['freight_price'] = df[freight_col].apply(safe_numeric) if freight_col else 0.0

df = df.dropna(subset=['qty','unit_price','freight_price']).reset_index(drop=True)


# === БАЗОВАЯ ПРИБЫЛЬ ===
baseline_profit, _, _, _ = evaluate_multiplier(df, 1.0)
print(f"Базовая прибыль (multiplier=1.0): {baseline_profit:,.2f}")


# === ГЕНЕТИЧЕСКИЙ АЛГОРИТМ ===
def make_individual():
    return random.uniform(MULT_MIN, MULT_MAX)

def mutate(ind):
    if random.random() < MUTPB:
        ind += random.gauss(0, MUT_STD)
        ind = max(MULT_MIN, min(MULT_MAX, ind))
    return ind

def crossover(a, b):
    if random.random() < CXPB:
        alpha = random.random()
        child1 = alpha * a + (1-alpha) * b
        child2 = alpha * b + (1-alpha) * a
        return child1, child2
    return a, b

def tournament_selection(pop, fits, k=TOURNAMENT_K):
    selected = []
    n = len(pop)
    for _ in range(n):
        aspirants = random.sample(range(n), k)
        best = max(aspirants, key=lambda i: fits[i])
        selected.append(pop[best])
    return selected


# === ЭВОЛЮЦИЯ ===
population = [make_individual() for _ in range(POP_SIZE)]
best_history = []

for gen in range(1, GENERATIONS+1):
    fits = [evaluate_multiplier(df, ind)[0] for ind in population]
    best_idx = int(np.argmax(fits))
    best_val = fits[best_idx]
    best_mult = population[best_idx]
    best_history.append((gen, best_mult, best_val))

    if gen % 10 == 0 or gen == 1 or gen == GENERATIONS:
        print(f"Gen {gen:3d}: best multiplier = {best_mult:.4f}, profit = {best_val:,.2f}")

    selected = tournament_selection(population, fits)
    next_pop = []

    # элитизм
    sorted_idx = sorted(range(len(fits)), key=lambda i: fits[i], reverse=True)
    elite_count = 2
    for i in range(elite_count):
        next_pop.append(population[sorted_idx[i]])

    # кроссовер и мутации
    while len(next_pop) < POP_SIZE:
        a = random.choice(selected)
        b = random.choice(selected)
        c1, c2 = crossover(a, b)
        next_pop.append(mutate(c1))
        if len(next_pop) < POP_SIZE:
            next_pop.append(mutate(c2))

    population = next_pop


# === РЕЗУЛЬТАТ ===
final_fits = [evaluate_multiplier(df, ind)[0] for ind in population]
pairs = list(zip(population, final_fits))
pairs_sorted = sorted(pairs, key=lambda x: x[1], reverse=True)
top20 = pairs_sorted[:20]

print("\nЛучшие 20 решений:")
for rank,(mult,profit) in enumerate(top20, start=1):
    print(f"{rank:2d}. multiplier={mult:.4f}, profit={profit:,.2f}")

best_multiplier, best_profit = pairs_sorted[0]
print(f"\nОптимальный множитель: {best_multiplier:.4f}")
print(f"Макс. прибыль: {best_profit:,.2f} (прирост {best_profit - baseline_profit:,.2f})")

# применим лучший множитель
best_total, p_new_arr, q_new_arr, profit_rows = evaluate_multiplier(df, best_multiplier)
df_out = df.copy()
df_out['suggested_unit_price'] = p_new_arr
df_out['suggested_qty'] = q_new_arr
df_out['suggested_row_profit'] = profit_rows

# сохраняем в CSV
out_path = "retail_price_suggested_prices.csv"
df_out.to_csv(out_path, index=False)
print(f"\nРезультаты сохранены в {out_path}")

Базовая прибыль (multiplier=1.0): 772,998.20
Gen   1: best multiplier = 1.9989, profit = 867,158.88
Gen  10: best multiplier = 2.0000, profit = 867,209.61
Gen  20: best multiplier = 2.0000, profit = 867,209.61
Gen  30: best multiplier = 2.0000, profit = 867,209.61
Gen  40: best multiplier = 2.0000, profit = 867,209.61
Gen  50: best multiplier = 2.0000, profit = 867,209.61
Gen  60: best multiplier = 2.0000, profit = 867,209.61
Gen  70: best multiplier = 2.0000, profit = 867,209.61
Gen  80: best multiplier = 2.0000, profit = 867,209.61

Лучшие 20 решений:
 1. multiplier=2.0000, profit=867,209.61
 2. multiplier=2.0000, profit=867,209.61
 3. multiplier=2.0000, profit=867,209.61
 4. multiplier=2.0000, profit=867,209.61
 5. multiplier=2.0000, profit=867,209.61
 6. multiplier=2.0000, profit=867,209.61
 7. multiplier=2.0000, profit=867,209.61
 8. multiplier=2.0000, profit=867,209.61
 9. multiplier=2.0000, profit=867,209.61
10. multiplier=2.0000, profit=867,209.61
11. multiplier=2.0000, profit=